In [ ]:
%pip install chromadb
%pip install ollama
%pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.8 MB/s 

In [ ]:
import os
import chromadb
import ollama # Added for the live LLM call
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pypdf import PdfReader

In [ ]:
# ID of our fine-tuned model on the Hugging Face Hub
EMBEDDING_MODEL_ID = "arvindcreatrix/bge-baes-my-qna-model"

# The specific LLM we want to use for generation via Ollama
GENERATOR_MODEL_ID = "llama2"

# Path to the PDF file you want to ask questions about
PDF_PATH = "/content/sample_explain.pdf"

# ChromaDB settings
CHROMA_PATH = "./rag_chroma_db"
COLLECTION_NAME = "rag_collection"

# The instruction prefix required by the BGE embedding model
INSTRUCTION_PREFIX = "Represent this sentence for searching relevant passages: "

In [ ]:
def get_llm_response(prompt: str) -> str:
    """
    Sends the prompt to a locally running Qwen2 model using Ollama.

    Args:
        prompt: The complete prompt to be sent to the LLM.

    Returns:
        The text response from the LLM.
    """
    try:
        # Send the prompt to the specified model
        response = ollama.chat(
            model=GENERATOR_MODEL_ID,
            messages=[{'role': 'user', 'content': prompt}]
        )
        # Extract and return the content from the response
        return response['message']['content']
    except Exception as e:
        print(f"\nAn error occurred while calling the LLM via Ollama: {e}")
        return "Error: Could not get a response from the language model. Is Ollama running?"


In [ ]:
def load_and_split_pdf(file_path: str) -> list[str]:
    """Loads a PDF, extracts text, and splits it into chunks."""
    print(f"Loading and splitting document: {file_path}")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"PDF file not found at: {file_path}")

    reader = PdfReader(file_path)
    text = "\n".join(page.extract_text() for page in reader.pages)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )

    chunks = text_splitter.split_text(text)
    print(f"Document split into {len(chunks)} chunks.")
    return chunks

def setup_rag_pipeline(pdf_path: str):
    """Sets up the entire RAG pipeline: embedding, chunking, and indexing."""
    print("--- Setting up RAG pipeline ---")

    # 1. Load and split the document
    chunks = load_and_split_pdf(pdf_path)

    # 2. Load the fine-tuned embedding model
    print(f"Loading embedding model: {EMBEDDING_MODEL_ID}")
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_ID)

    # 3. Setup ChromaDB
    print(f"Setting up ChromaDB at: {CHROMA_PATH}")
    chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
    collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)

    # 4. Embed and index the chunks
    print(f"Embedding and indexing {len(chunks)} chunks...")
    chunk_ids = [str(i) for i in range(len(chunks))]
    collection.add(
        ids=chunk_ids,
        documents=chunks,
        embeddings=embedding_model.encode(chunks).tolist()
    )

    print("--- RAG pipeline setup complete! ---")
    return collection, embedding_model

def ask_question(question: str, collection, embedding_model):
    """Handles the retrieval and generation for a single question."""

    # 1. Embed the user's question
    instructed_query = INSTRUCTION_PREFIX + question
    query_embedding = embedding_model.encode(instructed_query).tolist()

    # 2. Retrieve relevant context from ChromaDB
    retrieved_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=3
    )

    retrieved_context = "\n\n---\n\n".join(retrieved_results['documents'][0])

    # 3. Construct the prompt for the LLM
    prompt = f"""
    Context:
    {retrieved_context}

    Question: {question}

    Based on the context provided, please answer the question. If the context does not contain the answer, state that.
    Answer:
    """

    # 4. Get the response from the LLM
    answer = get_llm_response(prompt)
    return answer

In [ ]:
if __name__ == "__main__":
    # Create a dummy PDF for demonstration if one doesn't exist
    if not os.path.exists(PDF_PATH):
        print(f"Creating a dummy PDF at: {PDF_PATH}")
        try:
            from fpdf import FPDF
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10,
                "The theory of general relativity was proposed by Albert Einstein. It describes gravity as a fundamental property of spacetime. "
                "Isaac Newton developed the laws of motion and universal gravitation, which were the dominant theories for centuries before Einstein. "
                "In biology, the mitochondrion is known as the powerhouse of the cell, responsible for generating most of the cell's supply of adenosine triphosphate (ATP)."
            )
            pdf.output(PDF_PATH)
        except ImportError:
            print("Please install `fpdf` (pip install fpdf) to create the dummy PDF.")
            exit()

    # Setup the RAG pipeline (this will take a moment)
    collection, embedding_model = setup_rag_pipeline(PDF_PATH)

    # Start the interactive Q&A loop
    print("\n--- Ollama and Qwen2-0.5B Ready! ---")
    print("Enter your questions. Type 'exit' to quit.")
    while True:
        user_question = input("\nYour Question: ")
        if user_question.lower() == 'exit':
            break

        answer = ask_question(user_question, collection, embedding_model)
        print("\nAnswer:")
        print(answer)

    print("\n--- Session Ended ---")

--- Setting up RAG pipeline ---
Loading and splitting document: /content/sample_explain.pdf
Document split into 5 chunks.
Loading embedding model: arvindcreatrix/bge-baes-my-qna-model
Setting up ChromaDB at: ./rag_chroma_db
Embedding and indexing 5 chunks...
--- RAG pipeline setup complete! ---

--- Ollama and Qwen2-0.5B Ready! ---
Enter your questions. Type 'exit' to quit.

Your Question: heading of the document

Answer:
Question: What is the heading of the document?

Answer: The heading of the document is "PDF Bookmark Sample".

Your Question: objective of the pdf

Answer:
The objective of the PDF bookmark sample is to demonstrate the use of bookmarks in a PDF document. The sample includes a simple form with four fields, and a data file containing eight records. By default, the data file will produce a PDF file containing eight separate pages. However, by using the bookmark file, the same PDF can be generated with a separate pane displaying the bookmarks. This allows the user to navi

In [ ]:
if __name__ == "__main__":
    # Create a dummy PDF for demonstration if one doesn't exist
    if not os.path.exists(PDF_PATH):
        print(f"Creating a dummy PDF at: {PDF_PATH}")
        try:
            from fpdf import FPDF
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10,
                "The theory of general relativity was proposed by Albert Einstein. It describes gravity as a fundamental property of spacetime. "
                "Isaac Newton developed the laws of motion and universal gravitation, which were the dominant theories for centuries before Einstein. "
                "In biology, the mitochondrion is known as the powerhouse of the cell, responsible for generating most of the cell's supply of adenosine triphosphate (ATP)."
            )
            pdf.output(PDF_PATH)
        except ImportError:
            print("Please install `fpdf` (pip install fpdf) to create the dummy PDF.")
            exit()

    # Setup the RAG pipeline (this will take a moment)
    collection, embedding_model = setup_rag_pipeline(PDF_PATH)

    # Start the interactive Q&A loop
    print("\n--- Ollama and Qwen2-0.5B Ready! ---")
    print("Enter your questions. Type 'exit' to quit.")
    while True:
        user_question = input("\nYour Question: ")
        if user_question.lower() == 'exit':
            break

        answer = ask_question(user_question, collection, embedding_model)
        print("\nAnswer:")
        print(answer)

    print("\n--- Session Ended ---")

--- Setting up RAG pipeline ---
Loading and splitting document: /content/sample_explain.pdf
Document split into 5 chunks.
Loading embedding model: arvindcreatrix/bge-baes-my-qna-model
Setting up ChromaDB at: ./rag_chroma_db
Embedding and indexing 5 chunks...
--- RAG pipeline setup complete! ---

--- Ollama and Qwen2-0.5B Ready! ---
Enter your questions. Type 'exit' to quit.

Your Question: list all filename

Answer:
Sure! Here are the filenames listed in the context:

1. ap_bookmark.IFD
2. ap_bookmark.mdf
3. ap_bookmark.bmk
4. ap_bookmark.dat
5. ap_bookmark_doc.pdf

So, the answer to the question is:

[ap_bookmark.IFD, ap_bookmark.mdf, ap_bookmark.bmk, ap_bookmark.dat, ap_bookmark_doc.pdf]

Your Question: list all filename

Answer:
The following are the filenames listed in the context:

1. ap_bookmark.IFD
2. ap_bookmark.mdf
3. ap_bookmark.dat
4. ap_bookmark.bmk
5. ap_bookmark.pdf
6. ap_bookmark_doc.pdf

Your Question: exit

--- Session Ended ---


In [ ]:
# ID of our fine-tuned model on the Hugging Face Hub
EMBEDDING_MODEL_ID = "BAAI/bge-base-en-v1.5"

# The specific LLM we want to use for generation via Ollama
GENERATOR_MODEL_ID = "llama2"

# Path to the PDF file you want to ask questions about
PDF_PATH = "/content/sample_explain.pdf"

# ChromaDB settings
CHROMA_PATH = "./rag_chroma_db"
COLLECTION_NAME = "rag_collection"

# The instruction prefix required by the BGE embedding model
INSTRUCTION_PREFIX = "Represent this sentence for searching relevant passages: "

In [ ]:
if __name__ == "__main__":
    # Create a dummy PDF for demonstration if one doesn't exist
    if not os.path.exists(PDF_PATH):
        print(f"Creating a dummy PDF at: {PDF_PATH}")
        try:
            from fpdf import FPDF
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10,
                "The theory of general relativity was proposed by Albert Einstein. It describes gravity as a fundamental property of spacetime. "
                "Isaac Newton developed the laws of motion and universal gravitation, which were the dominant theories for centuries before Einstein. "
                "In biology, the mitochondrion is known as the powerhouse of the cell, responsible for generating most of the cell's supply of adenosine triphosphate (ATP)."
            )
            pdf.output(PDF_PATH)
        except ImportError:
            print("Please install `fpdf` (pip install fpdf) to create the dummy PDF.")
            exit()

    # Setup the RAG pipeline (this will take a moment)
    collection, embedding_model = setup_rag_pipeline(PDF_PATH)

    # Start the interactive Q&A loop
    print("\n--- Ollama and Qwen2-0.5B Ready! ---")
    print("Enter your questions. Type 'exit' to quit.")
    while True:
        user_question = input("\nYour Question: ")
        if user_question.lower() == 'exit':
            break

        answer = ask_question(user_question, collection, embedding_model)
        print("\nAnswer:")
        print(answer)

    print("\n--- Session Ended ---")

--- Setting up RAG pipeline ---
Loading and splitting document: /content/sample_explain.pdf
Document split into 5 chunks.
Loading embedding model: BAAI/bge-base-en-v1.5


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setting up ChromaDB at: ./rag_chroma_db
Embedding and indexing 5 chunks...
--- RAG pipeline setup complete! ---

--- Ollama and Qwen2-0.5B Ready! ---
Enter your questions. Type 'exit' to quit.

Your Question: explain the concept of book marking

Answer:
Bookmarking is the process of creating a digital bookmark or anchor point within a document or presentation, which allows users to quickly navigate to a specific section or location within the content. Bookmarks can be created manually by the user or automatically generated by the software, and they typically appear as a hyperlink or icon within the document or presentation.

In the context of this sample, bookmarking is used to create personalized bookmarks for different sections of a PDF document. The bookmarks are created using the -abmk command in the data file ap_bookmark.dat, which allows users to specify the target output device, bookmark file, and section for generation. The resulting bookmarks appear as hyperlinks within the PD